# SHIP: creating a model for a component

This section focuses on the definition of a component's model.

### Data convention: recall
We remind the convention by which data should be structured.

Input data should be a tensor of size (batch_size,nU), with nU being the number of units. For neurongroups in particular, the number of units is the number of components of the group. The output is expected to have the same format, as is the various group states (though there is more flexibility on the data processed internally).

For sake of simplicity, synapsegroups also perform what is part of the dendritic integration. In fact, if a synapsegroup connects a neurongroup A, having number of units NA = 3, and a neurongoup B of NB = 5, a synapsegroup should have a number of units nU = 3x5 = 15 components, but expects an input of shape (batch_size, NA) and delivers an output of shape (batch_size, NB). 
These are convention in place so to lower the size of data that must be shuttled between groups, but newer versions of SHIP might put in place a more flexible framework, separating synaptic functionality and dendritic integration. This would also simplify the creation of models intended for different funcitonalities other than the synaptic and neuronal ones.

### Preliminary action - adding folder

In [1]:
import sys
sys.path.append('C:\\') # to be edited by the user


## Generic model

### methods and properties of a model's class
To model each group, SHIP uses a **group** class, containing a scaffold of method intended to be edited easily.  We anticipate that, in order to facilitate certain internal operations, the **group** class is further divided in two distinct classes, the **neurongroup**, and **synapsegroup**, each applicable for either neurons or synapses.

These scaffold of methods that each group needs editing for are the following:
- `advance_timestep`, which is the heart of the model. This method is expected to deliver a time-step-dependent output each time it is called, generated according to a user-defined set of equations. This method can optionally intake a (time-step-dependent) input, and contain any number of internal states.
- `set_initial_state`, that is intended to perform any (non-automated) initialization operation to set up the model to be ready for the temporal evolution emulation via the `advance_timestep` method (a sort of zeroth-step, if you find it easier to see it like that!)
- `time_dep`, a method that precalculates the part of the model equations that explicitly depend on the time-step. This method has been put in place to enable one to reduce the reiterated calculations during the `advance_timestep` method.

Any additional method can indeed be included in the class definition, and called within the above methods.



It is also useful to explicitly map the variables and parameters. A dict property for the variables, `group.variables`, and one for the parameters, `group.parameters`, take this role. The dict can contain merely the keys (that match the desired variables and parameters, as typed within the equations). However, it is also reccommended to use, as dict values, the default values that variables/parameters can take. See an example below, for the 1st order leaky synapse:

In [2]:
from torch import rand
from SHIP.group import synapsegroup # useful as it pre-set the group to find the number of components as a function of source and target

class lS_1o(synapsegroup):
    variables = {'_I__': 0} # synapse current <--> internal state
    parameters = {'tau_alpha__': 8e-3, # temporal constant [s]
                  'w__': rand, # synaptic weight
                  'w_scale': 1} # global scaling factor

Note that the syntetic notation using underscores and generator functions can be employed here as well (see Tutorial_1 for more info).

### Automated initialization: information
We referred to non-automated initialization operations, and here recall some useful info about this.

During the network building stage, each group can accept user-defined values for a list of arguments matching the model's *variables* or *parameters*. 
- *variables* are assumed to be flexible containers, changing during the temporal evolution; they may match with the models' states. Those are set in the model when calling the `network.shallow_init` method, which is automatically done when calling `network.run`, but it can be avoided direclty calling `network.run_` or `network.run_no_out` (see Tutorial_2 for more information).
- *parameters* should remain static during the temporal evolution, and are always reset to the user-defined values when the user explicitly calls `network.init()` (but not when calling `network.run`).

Summarizing, `network.init` sets the user-provided values for the parameters of each group; `network.shallow_init` does the same, but for the user-provided variables. It is therefore not necessary to repeat these on the `set_initial_state` method for the group

## Example: 1st order leaky synapse

We reprise the 1st order leaky synapse to give an example as to how to code a component model.

As shown before, it is quite useful to explicitly report parameters and values within the reserved dicts, as these are automatically used to set the values provided during the network building stage.

At this point, we briefly digress to mention what to expect from this model.
A 1st order leaky synapse sees the variation of the current in time as follows:

    dI/dt = I / tau_alpha

Solving the differential equation, we obtain this solution:

    I(t) = I x exp(-t/tau_alpha)

Note that this solution can be easily transformed in a time-discrete equation, readily applicable to our model, by replacing the arbitrary time t with a time-step duration delta_t:

    I(t+delta_t) = I(t) x exp(-delta_t/tau_alpha)
    
We also need to take care of the synaptic input.
We can simply assume that, at the arrival of a pre-synaptic spike S, the current raises by the value w. This is an instantaneous event.

    I(t+delta_t) = I(t) x exp(-delta_t/tau_alpha) + wS

At this stage, we can further improve this model, by calculating the delta_t-dependent output, and we can easily do so by integrating this equation along the time-step duration (easily for this equation; more complex models might need some work). This lead to the following solution:

    I_integral(t) = I(t) x [tau_alpha x (1-exp(-delta_t/tau_alpha))]
    
Therefore, we can already define a stub of the `advance_timestep` method, that carries out these operations. Note that all variables and parameters are to be stored as the group properties, and thus these must be addressed as `self.name_of_the_property` for them to be properly set and remembered from time-step to time-step.

Note also that, in the current version of SHIP, the time-step size `dt` is already available as a property of each group (as are the `batch_size` and number of time-steps `nts`).


Let us write a first stub of the `advance_timestep` function.
We need the following properties:
- current I (a variable)
- current integral I_int 
- synaptic weight w (a parameter)
- 

In [3]:
from torch import (tensor,exp)
def advance_timestep(self,S = 0): # draft version
    # I: point-current (time-step dependent)
    # I_int: integral of the point current for each time-step
    
    self.I = self.I*tensor(-self.dt/self.tau_alpha).exp()  # <-- time-step current decrease
    
    self.I = self.I + S*self.w # <-- adding perturbation from S
    
    self.I_int = self.I* self.tau_alpha*(1-tensor(-self.dt/self.tau_alpha).exp()) # <-- calc output
    
    return self.I_int    

Already at this stage, we see how we can use `time_dep` to avoid repeating a few parts of the calculation above. We can use this to define two parameters, `alphaA` and `alphaB`, that take care of the time-step size dependency, as follows:

In [4]:
def time_dep(self): # here we pre-calculate part of the solution of the draft
    self.alphaA = tensor(-self.dt/self.tau_alpha).exp()
    self.alphaB = self.tau_alpha*(1-self.alphaA)

and we can modify `advance_timestep` so to use the new properties:

In [5]:
def advance_timestep(self,S = 0): # second draft
    # I: point-current (time-step dependent)
    
    # here we change the state I
    self.I = self.I* self.alphaA + S*self.w # both time decrease and perturbation effect 
    
    # and, since the output is not necessarily a group state, we do not store it internally
    # instead we just return the current integral value
    return self.I*self.alphaB

A new class including ``time_dep`` and `advance_timestep` as shown would already be functional.
However, it would not necessarily work for groups having a number of components greater than one, as the equations are not explicitly tailored to work with tensors of arbitrary size.

Below, we adjust ``advance_timestep`` in order to have a method that can work with 2D matrices of components, mapping a source and a target of arbitrary sizes.

We also include the post-synaptic integration of the currents targeting each individual post-synaptic neuron. This is not a feature of the synaptic model, but it is useful to put it here so to decrease the model's output size (though - this is an arbitrary choice!)

We note that each group have a few properties dedicated to store the size of their component pool, the one of the sources, and the one of the targets. Those are, respectively, `self.N` (own number of components), `self.Ns` (source number of components), and `self.Nt` (target number of components).

In [6]:
def advance_timestep(self,S = 0): # operative version
    # I: point-current (time-step dependent)
    
    # the input S is expected to be a tensor of size(batch_size,self.Ns); we need to expand it to
    # apply the spikes to each "column" of synapses targeting a different post-synaptic neuron
    # since we modify the method - we also propose a w_scale property, that act as a global scaling 
    # factor for the synaptic weights (default = 1, so to not do anything)
    
    local_input = S.unsqueeze(-1).expand(self.batch_size,self.Ns,self.Nt)*self.w*self.w_scale
    # see infos on the functions unsqueeze and expand in https://pytorch.org/docs/stable/torch.html
    
    # here we change the state I, that is assumed to be a tensor of size (self.Ns, self.Nt)
    self.I = self.I* self.alphaA +  local_input 

    # now we also edit the output, sending out the sum of the currents along each column of synapses
    # targeting each individual post-synaptic neuron
    return (self.I*self.alphaB).sum(dim=1) 

The latter is the method that can be employed as-is within a 1st-order leaky synapse group. There is no strict need to use the `set_initial_state` method (though we use it to check the datatype of certain parameters, but that's not as important as the rest of the calculations shown here).

To summarize:
- we have a model's state, the post-synaptic current `I`, that is a tensor of size (`Ns`,`Nt`); the state will be remembered time-step after time-step, as it is stored as a group property
- `time_dep` calculates a couple of parameters explicitly dependent on the time-step `dt`, namely `alphaA` and `alphaB`, also permanently available as group properties
- the state `I` varies according to the temporal dependency (multiplication of `I` by `alphaA`)
- the state is also updated according to the (reshaped) set of inputs `S`
- the output is the current integral along the time-step  (multiplication of `I` by `alphaB`), summed along the columns targeting each individual post-synaptic neuron

We note that our implementation calculates the variation of the current, then applies a spike perturbation, and only then calculates the integral (thus including the spike effect). These are arbitrary choices, that may or may not fit the intended scope of the end-user, which is tasked to check and possibly amend the group class where needed.

## Example: leaky-integrate and fire neuron

The LIF neuron is a well-known model that fits extremely well in the SHIP framework, due to the simplicity of its equations. Let's rapidly have a look at the equations.

The LIF neuron possesses an internal state, the membrane potential U, that varies through time as follows:

    dU/dt = U/tau_beta
    
The similarity with the 1st order leaky synapse is evident. Solving this 1st order ordinary differential equation, we get to the following solution:

    U(t+dt) = U(t)*exp(-dt/tau_beta)
    
Again, similarly to what happens with the leaky synapse model, we now include the effect of the synaptic current on the membrane potential:

    U(t+dt) = U(t)*exp(-dt/tau_beta) + S
    
(note that S is here expected to be the current integral along the temporal duration t -> t+dt)
    
We now need to indicate what happens when placing a potential threhsold ``thr``, after which the neuron fires and resets its potential. It is fairly easy using pseudocode:

    if U(t) > thr:
        U(t) = 0
        output = 1
    else:
        output = 0
    
We can define an initial model stub that carries out the operations above.
We already start defining the temporal constant `beta` as the exponential of ``(-dt/tau_beta)``.

In [7]:
from SHIP.group import neurongroup # so for the platform to know how to handle the internal com

class lifN(neurongroup):  # first draft
    
    variables = {'_u_': 0} # membrane potential <-- internal state
    parameters = {'thr': 1, # threshold potential - the neuron fires once u overcomes its value
                  'tau_beta_': 1e-3} # temporal constant [s]
    
    def time_dep(self):
        self.beta = tensor(-self.dt/self.tau_beta).exp()

    def advance_timestep(self,S=0):
        self.u = self.u*self.beta+S #<-- state variation
        spikes = (self.u-self.thr)<0 #<-- output determination
        self.u[spikes.detach()] = 0 #<-- state variation (reset after spike); 
        # detach() is here used to take care of possible issues during the training process (see torch manual for details)
        return spikes

This draft is already functional, as it already contains all the LIF model operations, structured within the available methods. 

However, to be fully integrated with SHIP (especially to deal with the training part), we need to modify it.

### Activator
In particular, SHIP needs to know where to locate the neuron activation function, so to apply its modified surrogate-gradient to allow back-propagation.
Therefore, it is advisable to use the reserved method, `activator`, to carry out this part of the model. See below how this is used for the lif model above:

In [8]:
class lifN(neurongroup):  # second draft    
    variables = {'_u_': 0}
    parameters = {'thr': 1,
                  'tau_beta_': 1e-3}
    
    def activator(self, arg):
        return arg<0
    
    def time_dep(self):
        self.beta = tensor(-self.dt/self.tau_beta).exp()

    def advance_timestep(self,S=0):
        self.u = self.u*self.beta+S #<-- state variation
        spikes = self.activator(self.u-self.thr) #<-- output determination
        self.u[spikes.detach()] = 0 #<-- state variation (reset after spike); 
        # detach() is here used to take care of possible issues during the training process (see torch manual for details)
        return spikes

Now, with this amendment, the neuron group can support all the PyTorch training functionalities enabled by SHIP. Of course, the opposite is also true (no activator -> no Torch training), and this formalism is not necessary if the user does not plan to train the system.

###  integrate

Another element that needs modification is the presence of the `integrate` property, which is a boolean tensor of same size as `u`, and enables the model to delegate the management of the "does the neuron integrate?" functionality (a.k.a. refractoriness).

SHIP allows to add the refractoriness functionality on neuron models, but it does so by applying necessary modifications to the `integrate` group's property. Therefore, to fully support this feature, the neuron models need to make proper use of this. 

Below, we show how:

In [9]:
def advance_timestep(self,S=0):
    self.u = self.integrate*self.u*self.beta+S 
    # state variation, also including the integrate selector
    # now we do not need to manually-reset u after spike!
    spikes = self.activator(self.u-self.thr) #<-- output determination
    self.integrate = ~spikes.detach() # but we need to determine when to integrate, like this!
    return spikes

Now, if a ``refractory(lifN)`` group is added to a **network**, it includes the refractory functionality, provided that the user sets the argument *refr_time*, i.e. the refractory time. To do so, SHIP will set the variable `self.integrate` to zero during each time-step that the corresponding neuron need not to integrate any input.

This is an arbitrary approach, and possibly not the most straightforward - but delegating the integrating role to just one variable (``integrate``) allows us to leave all the remaining bits of the model as they are, including having the untouched local input `S` in `advance_timestep` (in case other operations needs to be carried out).